In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel
import torch

In [ ]:
base_model = "meta-llama/Meta-Llama-3.1-8B-instruct"
fine_tuned_model = "llama-3.1-fine-tuned-model"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model)

In [ ]:
base_model_reload = AutoModelForCausalLM.from_pretrained(
        base_model,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
)

In [ ]:
model = PeftModel.from_pretrained(base_model_reload, fine_tuned_model)
model = model.merge_and_unload()

In [ ]:
## Testing 
text = "Gender: Female, Age: 55, Appointment Date: 2016-05-06T14:50:43Z, Reason: 2016-05-06T00:00:00Z, Hypertension: NO, Diabetes: NO, Alcoholism: NO, Handicap: NO, SMS Received: NO."
prompt = f"""
    Patient Information: Gender, Age, Appointment Date, Scheduled For, Medical Conditions, Hypertension, Diabetes, Alcoholism, Handicap, SMS Reminder Sent.
    
    Based on the information provided above, determine whether the patient will show up for their appointment.
    
    Important Notes:
    - Consider the all of the patient's inforamtion, including reminders received.
    - Focus on any patterns that might indicate a no-show, such as chronic conditions or missed reminders such as SMS as well as the difference in Appointment Date and Scheduled For.

    Your response should be in the following format:
    - Label: "Yes" if the patient will not show up, or "No" if they will show up.

    Example Output:
    - text: {text}
    - label: """.strip()

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

outputs = pipe(prompt, max_new_tokens=2, do_sample=True, temperature=0.1)
print(outputs[0]["generated_text"].split("label: ")[-1].strip())

In [ ]:
# Saving Locally
model_dir = "models/Llama-3.1-8B-Instruct-Patient-Attendence"
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)